# Обучение NLP для формирования умного кэшбека

In [1]:
import os
from itertools import chain
import multiprocessing

In [2]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec, FastText
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
import pandas as pd
import pymorphy2

In [3]:
num_cores = multiprocessing.cpu_count()

## Функция предобработки названия товара:
- Удаляем обычные цифры и различные скобки
- Приведение к нижнему регистру
- Токенизация(извлечение слов из текста)
- Морфологизация слов(приведение к нормальной форме)

In [4]:
morph = pymorphy2.MorphAnalyzer()

# Загрузка стоп-слов заранее, чтобы не загружать их при каждом вызове функции
stop_words = set(stopwords.words('russian'))

def preprocess_text(text):
    text = re.sub(r'\b\d+\b', '', text)
    text = re.sub(r'[^a-zA-Zа-яА-Я0-9]', ' ', text)
    text = re.sub(r'<[^>]+>', '', text)
    # Приведение текста к нижнему регистру
    text = text.lower()
    
    # Токенизация текста
    tokens = word_tokenize(text)
    
    preprocessed_text = [morph.parse(word)[0].normal_form for word in tokens if word not in stop_words]
    
    return preprocessed_text

Посмотрим как работает

In [5]:
text = "Красота и лицо9 99[denis]"
preprocessed_text = preprocess_text(text)

print(preprocessed_text)

['красота', 'лицо9', 'denis']


### Формирование датасета для обучения модели
Для обучения модели Word2Vec я взял датасет от ozon, в котором содержатся нужные для обучения колонки: Название товара, Категория n уровня, где n &#8712; [1...4]

In [6]:
onlyfiles = ['data_cat/dat2/' + f for f in os.listdir('data_cat/dat2') if os.path.isfile(os.path.join('data_cat/dat2', f))]

In [7]:
def process_dataframe(file):
    df = pd.read_excel(file)
    df['Название товара'] = df['Название товара'].astype(str)
    df.dropna(subset=['Название товара'], inplace=True)
    df['Processed_Name'] = df['Название товара'].map(preprocess_text)
    df['Категория 2 уровня'] = df['Категория 2 уровня'].astype(str)
    df.dropna(subset=['Категория 2 уровня'], inplace=True)
    df['Cat2'] = df['Категория 2 уровня'].map(preprocess_text)
    df['Категория 3 уровня'] = df['Категория 3 уровня'].astype(str)
    df.dropna(subset=['Категория 3 уровня'], inplace=True)
    df['Cat3'] = df['Категория 3 уровня'].map(preprocess_text)
    df['Категория 4 уровня'] = df['Категория 4 уровня'].astype(str)
    df.dropna(subset=['Категория 4 уровня'], inplace=True)
    df['Cat4'] = df['Категория 4 уровня'].map(preprocess_text)
    df['Категория 1 уровня'] = df['Категория 1 уровня'].astype(str)
    df.dropna(subset=['Категория 1 уровня'], inplace=True)
    df['Cat1'] = df['Категория 1 уровня'].map(preprocess_text)
    df['Cat'] = df['Cat1'] + df['Cat2'] + df['Cat3'] + df['Cat4']
    return df.apply(lambda row: row['Cat'] + [row['Категория 1 уровня']], axis=1).tolist()

pool = multiprocessing.Pool(processes=num_cores)

# Использование map для параллельной обработки данных
res = pool.map(process_dataframe, onlyfiles)

# Закрытие пула процессов
pool.close()
pool.join()

In [8]:
rs = list(chain.from_iterable(res))

Посмотрим как выглядит датасет, подаваемы в модель

In [9]:
rs

[['тв',
  'аудио',
  'игровой',
  'приставка',
  'аксессуар',
  'tv',
  'audio',
  'игровой',
  'приставка',
  'tv',
  'audio',
  'игровой',
  'приставка',
  'ТВ и аудио'],
 ['тв',
  'аудио',
  'игровой',
  'приставка',
  'аксессуар',
  'tv',
  'audio',
  'игровой',
  'приставка',
  'tv',
  'audio',
  'игровой',
  'приставка',
  'ТВ и аудио'],
 ['книга',
  'современный',
  'печатный',
  'книга',
  'современный',
  'печатный',
  'книга',
  'современный',
  'печатный',
  'книга',
  'Книги'],
 ['тв',
  'аудио',
  'игровой',
  'приставка',
  'аксессуар',
  'tv',
  'audio',
  'игровой',
  'приставка',
  'tv',
  'audio',
  'игровой',
  'приставка',
  'ТВ и аудио'],
 ['товар',
  'мама',
  'ребёнок',
  'игрушка',
  'развлекательный',
  'игрушка',
  'антистрессовый',
  'игрушка',
  'Товары для мам и детей'],
 ['тв',
  'аудио',
  'игровой',
  'приставка',
  'аксессуар',
  'tv',
  'audio',
  'игровой',
  'приставка',
  'tv',
  'audio',
  'игровой',
  'приставка',
  'ТВ и аудио'],
 ['красота',
  '

Обучаем модель на данных. Параметры старался подбирать с помощью анализа: слова близкие по семантике должны быть на косинусовом расстоянии не менее чем на 70 процентов, дальние - не более 40

In [67]:
model = Word2Vec(rs, vector_size=50, epochs=30, window=4, min_count=1, workers=os.cpu_count(), sg=1)

Посмотрим на сколько близки одно и тоже слово в разных регистрах

In [78]:
vector1 = model.wv['одежда']  # Получение векторного представления строки 1
vector2 = model.wv['Одежда']  # Получение векторного представления строки 2

# Рассчет косинусного расстояния
similarity = model.wv.cosine_similarities(vector1, [vector2])[0]

In [79]:
similarity

0.7969132

Хороший результат и это он показывает при не особо большом датасете

In [55]:
model.save('word2vec')